# Tests for `get_margins` and `get_a_margin`

In [ ]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = 'SNP'

# Add `src` to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path
from pathlib import Path
if 'src' not in Path.cwd().parts:
    src_path = str(Path(get_path('purelib')) / '_src.pth')
    with open(src_path, 'w') as f:
        f.write(str(Path.cwd() / 'src\n'))

# Start the Jupyter loop
from ib_insync import util, IB
util.startLoop()

In [ ]:
# Set the root
from from_root import from_root
ROOT = from_root()

from utils import Vars
_vars = Vars(MARKET)
PORT = _vars.PORT
PAPER = _vars.PAPER 
OPT_COLS = _vars.OPT_COLS[0]
DATAPATH = ROOT / 'data' / MARKET

In [ ]:
# Imports 
from utils import get_pickle, get_order_pf, qualify_conIds, clean_ib_util_df, get_dte
from ib_insync import MarketOrder
import asyncio
import pandas as pd
import numpy as np

## Fix `get_a_margin` function

In [ ]:
from typing import Union
from utils import BAR_FORMAT, get_lots, chunk_me, clean_ib_util_df
from ib_insync import Contract, MarketOrder
from tqdm import tqdm
from loguru import logger

In [ ]:
def clean_a_margin(wif, conId):
    """Clean up wif margins"""

    d = dict()

    df = util.df([wif])[["initMarginChange", "maxCommission",
                                "commission"]].astype('float')

    df = df.assign(
        comm=df[["commission", "maxCommission"]].min(axis=1),
        margin=df.initMarginChange,
        conId = conId
    )

    # Correct unrealistic margin and commission
    df = df.assign(conId=conId,
        margin=np.where(df.initMarginChange > 1e7, np.nan, df.initMarginChange),
        comm=np.where(df.comm > 1e7, np.nan, df.comm))

    d[conId] = df[['margin', 'comm']].iloc[0].to_dict()

    return d

In [ ]:
def to_list(data):
    """Converts any iterable to a list, and non-iterables to a list with a single element.

    Args:
        data: The data to be converted.

    Returns:
        A list containing the elements of the iterable, or a list with the single element if the input is not iterable.
    """

    try:
        return list(data)
    except TypeError:
        return [data]

In [ ]:
async def get_a_margin(ib: IB, 
                       contract,
                       order: Union[MarketOrder, None]=None,
                       lots_path: Path=None,
                       ACTION: str='SELL',
                       ):
    
    """[async] Gets a margin"""

    if lots_path: # For NSE
        lot_size = get_lots(contract, lots_path)
    else:
        lot_size = get_lots(contract)

    if not order: # Uses ACTION instead of order
        order = MarketOrder(ACTION, lot_size)

    def onError(reqId, errorCode, errorString, contract):
        logger.error(f"{contract.localSymbol} with reqId: {reqId} has errorCode: {errorCode} error: {errorString}")

    ib.errorEvent += onError
    wif = await ib.whatIfOrderAsync(contract, order)
    ib.errorEvent -= onError
    logger.remove()

    try:
        output = clean_a_margin(wif, contract.conId)
    except KeyError:
        output = {contract.conId: {
                  'margin': None,
                  'comm': None}}
        
        logger.error(f"{contract.localSymbol} has no margin and commission")

    output[contract.conId]['lot_size'] = lot_size
    return output


In [ ]:
async def get_margins(port: int, 
                      contracts: Union[pd.Series, list, Contract],
                      orders: Union[pd.Series, list, MarketOrder, None]=None,                      
                      lots_path: Path=None,
                      ACTION: str='SELL', 
                      chunk_size: int=100,
                      CID: int=0):
    """
    [async] Gets margins for options contracts with `orders` or `ACTION`

    Parameters
    ---
    contracts: df with `contract` field | list
    order: list of `MarketOrder` | `None` requires an `ACTION`
    ACTION: `BUY` or `SELL` needed if no `orders` are provided. Defaults to `SELL`

    """
    
    opt_contracts = to_list(contracts)

    pbar = tqdm(total=len(opt_contracts),
                    desc="Getting margins:",
                    bar_format = BAR_FORMAT,
                    ncols=80,
                    leave=True,
                )
    
    # prepare orders
    if orders:
        orders = to_list(orders)

        if len(orders) == 1: # single order
            orders = orders*len(opt_contracts)

    else:
        orders = [None]*len(opt_contracts)
    
    results = list()

    df_contracts = clean_ib_util_df(opt_contracts)

    df_contracts = df_contracts.assign(conId=[c.conId for c in opt_contracts], order = orders)\
                                .set_index('conId')

    cos = list(zip(df_contracts.contract, df_contracts.order))

    chunks = chunk_me(cos, chunk_size)

    with await IB().connectAsync(port=port, clientId=CID) as ib:
        
        for cts in chunks:

            tasks = [asyncio.create_task(get_a_margin(ib=ib, 
                                                        contract=contract,
                                                        order=order,
                                                        ACTION=ACTION,
                                                        lots_path=lots_path), 
                                                        name= contract.localSymbol) 
                        for contract, order in cts]        


            margin = await asyncio.gather(*tasks)

            results += margin
            pbar.update(len(cts))
            pbar.refresh()

    flat_results ={k: v for r in results for k, v in r.items()}
    df_mgncomm = pd.DataFrame(flat_results).T
    df_out = df_contracts.join(df_mgncomm).reset_index()

    pbar.close()

    return df_out

In [ ]:
# contract types

lots_path = DATAPATH.parent / 'nse' / 'lots.pkl'
# nse_option = get_pickle(DATAPATH.parent / 'nse' / 'df_qualified_calls.pkl').contract.sample(1).iloc[0]
# nse_stock = get_pickle(DATAPATH.parent / 'nse' / 'unds.pkl').get('RELIANCE')
# nse_index = get_pickle(DATAPATH.parent / 'nse' / 'unds.pkl').get('NIFTY50')

snp_option = get_pickle(DATAPATH.parent / 'snp' / 'df_qualified_calls.pkl').contract.sample(1).iloc[0]
snp_stock = get_pickle(DATAPATH.parent / 'snp' / 'unds.pkl').get('IBM')
snp_index = get_pickle(DATAPATH.parent / 'snp' / 'unds.pkl').get('VIX')


In [ ]:
# many contracts
from ib_insync import Index

# many_nse_options = get_pickle(DATAPATH.parent / 'nse' / 'df_qualified_calls.pkl').qualified_opts.sample(3)
# many_nse_stocks = [get_pickle(DATAPATH.parent / 'nse' / 'unds.pkl').get(s) for s in ['RELIANCE', 'PNB']]
# many_nse_indexes = [get_pickle(DATAPATH.parent / 'nse' / 'unds.pkl').get(i) for i in ['NIFTY50', 'BANKNIFTY']]

many_snp_options = get_pickle(DATAPATH.parent / 'snp' / 'df_qualified_calls.pkl').contract.sample(5)
many_snp_stocks = [get_pickle(DATAPATH.parent / 'snp' / 'unds.pkl').get(s) for s in ['INTC', 'TSLA', 'NVDA']]
many_snp_indexes = [v for k, v in get_pickle(DATAPATH.parent / 'snp' / 'unds.pkl').items() if isinstance(v, Index)]

In [ ]:
# contracts = many_snp_options
# contract = contracts.to_list()[0]


In [ ]:
from utils import get_order_pf, qualify_conIds
order, pf = asyncio.run(get_order_pf(PORT))

In [ ]:
contracts = asyncio.run(qualify_conIds(PORT, pf.conId.to_list()))

In [ ]:
margins = asyncio.run(get_margins(PORT, contracts))

In [ ]:
margins